In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pprint
import time
import datetime
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import unicodedata
import re

from selenium import webdriver

def get_soup(url):

    header = {
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36Name"
    }
    time.sleep(1)
    content = requests.get(url , headers = header).text

    soup = BeautifulSoup(content, "html.parser")
    
    return soup



In [3]:
def cleanJD(data):
    cleaned_data =  re.split(r'<.*?>', str(data))
    cleaned_data = [word.strip() for word in cleaned_data if word]
    cleaned_data = [unicodedata.normalize("NFKD", line) for line in cleaned_data]

    return cleaned_data

def getJobInfo(url):
    keys = ['jobtitle', 'date_posted', 'contract_type', 'job_id', 'job_location', 'job_description']
    dct = dict.fromkeys(keys, None)
    
    try:
        soup = get_soup(url)
        dct['jobtitle']=soup.find('h1',{'class':'h2-style'}).text
        dct['date_posted']=soup.find('span',{'class':"job-date job-info icon-time with-text"}).text
        dct['contract_type'] = soup.find('span',{'class':"icon-contract with-text"}).text
        dct['job_id'] = soup.find('span',{'class':"job-id job-info icon-job-id with-text"}).text
        dct['job_location'] = soup.find('span',{'class':"job-location job-info white-text"}).text
        raw_jd = soup.find('div',{'class':'ats-description'})
        dct['job_description'] = cleanJD(raw_jd)
    except:
        pass
    
    return dct

In [38]:

jobpostings = []
browser = webdriver.Chrome()
#continue to get search listings if month = querymonth or later
# browser.implicitly_wait(10)


url = {URL}

try:
    browser.get(url)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'search-results-list')))
#         time.sleep(5)
    results = browser.find_element_by_id('search-results-list').find_elements_by_tag_name('li')    
   
except:
    print("Error reading {}".format(url))

time.sleep(1)


def getjobList(results):
    jobpostings = []
    
    try:
        for result in results[1:]:
            dct={}
            details = result.find_element_by_tag_name('a')
            dct['url'] = details.get_attribute('href')
            dct['ID']= details.get_attribute('data-job-id')#can remove
            dct['job title'] = result.find_element_by_class_name('h3-style').text
            dct['location'] = result.find_element_by_class_name('job-location').text
            dct['date posted'] = result.find_element_by_class_name('job-date-posted').text
            dct.update(getJobInfo(dct['url']))
            jobpostings.append(dct)
    except:
        pass
    
    return jobpostings

#get first page results
jobpostings.extend(getjobList(results))

#get max number of pages
max_page = browser.find_element_by_class_name('pagination-current').get_attribute('max')

#parse pages 2 up to max-page
i=2
while i<=int(max_page):
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'next')))
    next_page_btn = browser.find_element_by_class_name('next')
    next_page_btn.click()
#
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'search-results-list')))
    results = browser.find_element_by_id('search-results-list').find_elements_by_tag_name('li')    
    jobpostings.extend(getjobList(results))
    print('read page {}'.format(i))
    i+=1
    

read page 2
